# 1.RAG知识库构建的用户提问之前：文本分片、文本索引Embedding模型向量化、向量数据库存储

In [10]:
from typing import List
import pdfplumber

def split_into_chunks(pdf_file: str) -> List[str]:
    # 初始化文本内容
    content = ""

    # 使用pdfplumber读取PDF文件
    with pdfplumber.open(pdf_file) as pdf:
        # 遍历所有页面并提取文本
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                content += page_text + "\n\n"  # 页面之间添加空行分隔

    # 按空行分割内容为块
    return [chunk.strip() for chunk in content.split("\n\n") if chunk.strip()]
# 读取PDF文件并分割
chunks = split_into_chunks("基于改进EfficientNet的番茄叶片病害识别.pdf")

# 打印每个文本块
for i, chunk in enumerate(chunks):
    print(f"[{i}] {chunk}\n")

[0] 本科毕业论文（设计）
基于改进 EfficientNet 的番茄叶片病害识别
院 部： 信息与智能工程学院
姓 名： 郭兆国
学 号： 20221001714
专 业： 人工智能
年 级： 2022级
指导教师： 杨云
职 称： 教授
完成日期： 2025年8月

[1] 云南经济管理学院毕业论文（设计）原创性及知识产权声明
本人郑重声明：所呈交的论文（设计）是本人在指导教师的指导下独立进行研究
所取得的成果。除了文中特别加以标注引用的内容外，本论文（设计）不包括任何其
他人或集体已经发表或撰写的成果作品。本人完全意识到本声明的法律后果由本人承
担
特此声明
毕业论文（设计）作者签名：
年 月 日
关于毕业论文（设计）使用和授权的说明
本毕业论文（设计）作者完全了解云南经济管理学院有关保留、使用毕业论文
（设计）的规定，即：学校有权保留并向国家有关部门或机构送交毕业论文（设计）
的复印件和电子版；允许毕业论文（设计）被查阅 和借阅。本人授权云南经济管理学
院可以将本毕业论文（设计）的全部内容或部分内容编入有 关数据库进行检索，可以
采用影印、缩印或扫描等复制手段保存和汇编本毕业论文（设计）。
毕业论文（设计）作者签名： 指导教师签名：
年 月 日 年 月 日

[2] 摘 要
目前在番茄叶片病害识别领域，存在资源受限场景下的高精度识别需求。为解决
该需求，本文基于 EfficientNetB0架构，提出了融合动态通道注意力（DCA）、可分
离扩张卷积（SDC）和通道注意力（CAN）的改进模型 DSC-EfficientNetB0。本实验
采用的数据集来源于 PlantVillage数据库，包含 10种番茄叶片病害疾病。在数据预处
理阶段进行了数据平衡、增强、划分以及归一化等操作，并在后续实验训练中采用了
迁移学习和混合精度训练策略。最终，对比实验展示出该模型准确率为 99.78%，参数
量 4.26M，FLOPs为 0.7974G，推理时间 2.53s。在准确率与损失曲线波动小、收敛快
的良好状态下，既保持了高精度识别又明显地降低了计算资源，实现了精度与效率的
良好平衡，远优于原版 EfficientNetB0、ResNet50、MobileNetV3Small 等主流模型。随
后，消融实验与混淆矩阵证明了 DCA、SDC和 CAN的协同增效性。综上，该研究

In [31]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("./text2vec-base-chinese")

def embed_chunk(chunk: str) -> List[float]:
    embedding = embedding_model.encode(chunk, normalize_embeddings=True)
    return embedding.tolist()


embedding = embed_chunk("测试内容")
print(len(embedding))
print(embedding)

No sentence-transformers model found with name ./text2vec-base-chinese. Creating a new one with mean pooling.


768
[0.02680548094213009, 0.008382001891732216, 0.0003433947858866304, 0.007298970129340887, 0.054333124309778214, -0.053255870938301086, 0.0013655673246830702, -0.001318164519034326, -0.03671123459935188, 0.07188183069229126, -0.0072706639766693115, -0.00705305952578783, 0.04253280535340309, -0.03675275668501854, -0.054750557988882065, -0.009598523378372192, 0.017105575650930405, 0.059153616428375244, -0.033349987119436264, 0.06237656623125076, -0.004888517782092094, -0.03453952446579933, -0.07407603412866592, 0.04422193020582199, 0.010516912676393986, -0.03707785904407501, -0.027029842138290405, 0.03830360993742943, 0.021282507106661797, -0.011811514385044575, -0.005408767145127058, 0.0026590183842927217, -0.02329857647418976, 0.05299096554517746, 0.005149467382580042, 0.029624201357364655, -0.030809693038463593, -0.017856137827038765, 0.042446013540029526, -0.007692311424762011, -0.010638120584189892, 0.032108649611473083, -0.06592468172311783, -0.012100937776267529, 0.0068146362900

In [32]:
embeddings = [embed_chunk(chunk) for chunk in chunks]

print(len(embeddings))
print(embeddings[0])

53
[0.008454954251646996, 0.03177478909492493, 0.0305020734667778, 0.03740982711315155, 0.022135233506560326, -0.034457940608263016, 0.032781679183244705, 0.033929526805877686, -0.020176883786916733, -0.02119397185742855, -0.016814373433589935, -0.020488079637289047, -0.01978662796318531, -0.013938120566308498, -0.06135847419500351, 0.01562415435910225, 0.007220783736556768, 0.028864990919828415, -0.032593321055173874, 0.04055970534682274, -0.017099957913160324, 0.052967704832553864, 0.024328991770744324, -0.02360982820391655, 0.0022618286311626434, -0.015850581228733063, -0.037585772573947906, -0.023793423548340797, -0.0054415566846728325, -0.00349531346000731, 0.027359798550605774, -0.044305335730314255, 0.0007799045997671783, 0.04864998906850815, -0.03694365546107292, 0.027488945052027702, -0.017570042982697487, -0.03301914781332016, 0.052537404000759125, -0.02768436260521412, 0.02618928626179695, 0.0033328065183013678, 0.02145726978778839, -0.059192392975091934, -0.0340155325829982

In [37]:
import chromadb

chromadb_client = chromadb.PersistentClient('./data')#将数据存在在指定文件中(.EphemeralClient则是不存放每次运行后就删除数据)
chromadb_collection = chromadb_client.get_or_create_collection(name="default")

def save_embeddings(chunks: List[str], embeddings: List[List[float]]) -> None:
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        chromadb_collection.add(
            documents=[chunk],
            embeddings=[embedding],
            ids=[str(i)]
        )

save_embeddings(chunks, embeddings)

# 2.RAG知识库构建的用户提问之后：召回、重排、生成

## 召回阶段，快速匹配与问题相似的片段

In [38]:
def retrieve(query: str, top_k: int) -> List[str]:
    query_embedding = embed_chunk(query)
    results = chromadb_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results['documents'][0]

query = "这篇论文的DSC-EfficientNetB0模型有什么特点和优势？"
retrieved_chunks = retrieve(query, 5)

for i, chunk in enumerate(retrieved_chunks):
    print(f"[{i}] {chunk}\n")

[0] 4 基于 DSC-EfficientNetB0的番茄叶片病害识别模型
面对传统高精度模型因计算复杂度居高不下，以及轻量化模型普遍在训练不稳定
和识别精度不足的问题，本文在前期相关研究的基础上，提出了一种融合动态通道注
意力（DCA）、可分离扩张卷积（SDC）和通道注意力（CAN）模块的改进型 DSC-
EfficientNetB0模型。该模型兼顾了网络结构的轻量化设计和计算高效性，显著提高
对番茄叶片复杂病害特征的提取与识别能力。本章将详具体阐述 DSC-EfficientNetB0
模型的整体架构设计和关键模块的运行原理，及其在番茄叶片病害识别任务中的实际
应用效果。
4.1 DSC-EfficientNetB0模型设计思路
4.1.1 EfficientNetB0作为主干网络的选择
本研究所提出的 DSC-EfficientNetB0 模型之所以选取 EfficientNetB0 作为主干网
络，是因为EfficientNet系列的核心在于采用复合缩放策略。通过同步优化网络深度、
通道宽度和图像分辨率，在参数量规模和实际计算量之间达成了更精细的平衡。对比
传统卷积神经网络，EfficientNet借助复合缩放机制，能在显著降低计算成本的前提下，
高效完成高质量的特征学习任务。
其中，在同 EfficientNet系列中，EfficientNetB0属于参数量少、计算复杂度最低
的轻量级版本，且依旧在图像识别等任务中保持优异的识别精度。正因如此，在边缘
设备这类资源受限的的实际环境中，EfficientNetB0的优势尤为突出，能对于番茄叶
片病害识别任务做到精准契合。
4.1.2 结构改进的必要性
虽然 EfficientNetB0已经展现出了足够的优势，但在番茄叶片病害识别任务中，
由于番茄叶片病害具有相似性和复杂性，尤其是在细粒度特征的提取上，如果能够继
续改进优化 EfficientNetB0，增强其特征提取能力，或继续降低模型参数量，减少资
源消耗，那么它将更好的适配低边缘设备。
为此，本研究具体的改进方向为：
（1） 增强模型对局部细节特征的提取能力，提升对不同病害的识别表现。
（2） 模型以更低的参数量与计算复杂度在低资源环境下实现高精度识别。
4.1.3 引入新模块
由本文的第二章介绍可知迁移学习在深度学习领域中发挥着重要作用，因此在迁
移

## 重排，精细的匹配与问题最接近的片段

In [41]:
from sentence_transformers import CrossEncoder

def rerank(query: str, retrieved_chunks: List[str], top_k: int) -> List[str]:
    cross_encoder = CrossEncoder('./cross-encodermmarco-mMiniLMv2-L12-H384-v1')
    pairs = [(query, chunk) for chunk in retrieved_chunks]
    scores = cross_encoder.predict(pairs)

    scored_chunks = list(zip(retrieved_chunks, scores))
    scored_chunks.sort(key=lambda x: x[1], reverse=True)

    return [chunk for chunk, _ in scored_chunks][:top_k]

reranked_chunks = rerank(query, retrieved_chunks, 3)

for i, chunk in enumerate(reranked_chunks):
    print(f"[{i}] {chunk}\n")

[0] 4 基于 DSC-EfficientNetB0的番茄叶片病害识别模型
面对传统高精度模型因计算复杂度居高不下，以及轻量化模型普遍在训练不稳定
和识别精度不足的问题，本文在前期相关研究的基础上，提出了一种融合动态通道注
意力（DCA）、可分离扩张卷积（SDC）和通道注意力（CAN）模块的改进型 DSC-
EfficientNetB0模型。该模型兼顾了网络结构的轻量化设计和计算高效性，显著提高
对番茄叶片复杂病害特征的提取与识别能力。本章将详具体阐述 DSC-EfficientNetB0
模型的整体架构设计和关键模块的运行原理，及其在番茄叶片病害识别任务中的实际
应用效果。
4.1 DSC-EfficientNetB0模型设计思路
4.1.1 EfficientNetB0作为主干网络的选择
本研究所提出的 DSC-EfficientNetB0 模型之所以选取 EfficientNetB0 作为主干网
络，是因为EfficientNet系列的核心在于采用复合缩放策略。通过同步优化网络深度、
通道宽度和图像分辨率，在参数量规模和实际计算量之间达成了更精细的平衡。对比
传统卷积神经网络，EfficientNet借助复合缩放机制，能在显著降低计算成本的前提下，
高效完成高质量的特征学习任务。
其中，在同 EfficientNet系列中，EfficientNetB0属于参数量少、计算复杂度最低
的轻量级版本，且依旧在图像识别等任务中保持优异的识别精度。正因如此，在边缘
设备这类资源受限的的实际环境中，EfficientNetB0的优势尤为突出，能对于番茄叶
片病害识别任务做到精准契合。
4.1.2 结构改进的必要性
虽然 EfficientNetB0已经展现出了足够的优势，但在番茄叶片病害识别任务中，
由于番茄叶片病害具有相似性和复杂性，尤其是在细粒度特征的提取上，如果能够继
续改进优化 EfficientNetB0，增强其特征提取能力，或继续降低模型参数量，减少资
源消耗，那么它将更好的适配低边缘设备。
为此，本研究具体的改进方向为：
（1） 增强模型对局部细节特征的提取能力，提升对不同病害的识别表现。
（2） 模型以更低的参数量与计算复杂度在低资源环境下实现高精度识别。
4.1.3 引入新模块
由本文的第二章介绍可知迁移学习在深度学习领域中发挥着重要作用，因此在迁
移

## 大模型生成

In [47]:
import requests
from typing import List

def generate(query: str, chunks: List[str]) -> str:
    # 先拼接 chunks 为字符串（单独处理，避免在 f-string 表达式中包含反斜杠）
    context = "\n\n".join(chunks)
    
    # 用变量 context 替代直接在 f-string 中拼接，避免反斜杠导致的语法错误
    prompt = f"""你是一位了解DSC-EfficientNetB0模型番茄叶片病害识别的知识助手，请根据用户的问题和下列片段生成准确的回答。

用户问题: {query}

相关片段:
{context}

请基于上述内容作答，不要编造信息。"""

    print(f"{prompt}\n\n---\n")

    # 后续 Ollama 调用代码不变...
    OLLAMA_API_URL = "http://localhost:11434/api/chat"
    MODEL_NAME = "deepseek-r1:1.5b"
    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }

    try:
        response = requests.post(
            OLLAMA_API_URL,
            json=payload,
            headers={"Content-Type": "application/json"},
            timeout=60
        )
        response.raise_for_status()
        result = response.json()
        return result["message"]["content"]
    except requests.exceptions.RequestException as e:
        return f"调用 Ollama 模型失败：{str(e)}"

answer = generate(query, reranked_chunks)
print(answer)

你是一位了解DSC-EfficientNetB0模型番茄叶片病害识别的知识助手，请根据用户的问题和下列片段生成准确的回答。

用户问题: 这篇论文的DSC-EfficientNetB0模型有什么特点和优势？

相关片段:
4 基于 DSC-EfficientNetB0的番茄叶片病害识别模型
面对传统高精度模型因计算复杂度居高不下，以及轻量化模型普遍在训练不稳定
和识别精度不足的问题，本文在前期相关研究的基础上，提出了一种融合动态通道注
意力（DCA）、可分离扩张卷积（SDC）和通道注意力（CAN）模块的改进型 DSC-
EfficientNetB0模型。该模型兼顾了网络结构的轻量化设计和计算高效性，显著提高
对番茄叶片复杂病害特征的提取与识别能力。本章将详具体阐述 DSC-EfficientNetB0
模型的整体架构设计和关键模块的运行原理，及其在番茄叶片病害识别任务中的实际
应用效果。
4.1 DSC-EfficientNetB0模型设计思路
4.1.1 EfficientNetB0作为主干网络的选择
本研究所提出的 DSC-EfficientNetB0 模型之所以选取 EfficientNetB0 作为主干网
络，是因为EfficientNet系列的核心在于采用复合缩放策略。通过同步优化网络深度、
通道宽度和图像分辨率，在参数量规模和实际计算量之间达成了更精细的平衡。对比
传统卷积神经网络，EfficientNet借助复合缩放机制，能在显著降低计算成本的前提下，
高效完成高质量的特征学习任务。
其中，在同 EfficientNet系列中，EfficientNetB0属于参数量少、计算复杂度最低
的轻量级版本，且依旧在图像识别等任务中保持优异的识别精度。正因如此，在边缘
设备这类资源受限的的实际环境中，EfficientNetB0的优势尤为突出，能对于番茄叶
片病害识别任务做到精准契合。
4.1.2 结构改进的必要性
虽然 EfficientNetB0已经展现出了足够的优势，但在番茄叶片病害识别任务中，
由于番茄叶片病害具有相似性和复杂性，尤其是在细粒度特征的提取上，如果能够继
续改进优化 EfficientNetB0，增强其特征提取能力，或继续降低模型参数量，减少资
源消耗，那么它将更好的适配低边缘设备。
为此，本研究具体的改进方向为：
（1） 增强模型对局部细

In [ ]:

from dotenv import load_dotenv
from google import genai

load_dotenv()
google_client = genai.Client()

def generate(query: str, chunks: List[str]) -> str:
    prompt = f"""你是一位知识助手，请根据用户的问题和下列片段生成准确的回答。

用户问题: {query}

相关片段:
{"\n\n".join(chunks)}

请基于上述内容作答，不要编造信息。"""

    print(f"{prompt}\n\n---\n")

    response = google_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    return response.text

answer = generate(query, reranked_chunks)
print(answer)